# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [44]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    vocab = sorted(word_counts, key = word_counts.get, reverse = True)

    vocab_to_int = {word: i for i, word in enumerate(vocab, 0)}
    int_to_vocab = dict(enumerate(vocab, 0))
    #print(vocab_to_int)
    #print(int_to_vocab)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [45]:
import string

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokens = {
        '.':'||Period||',
        ',':'||Comma||',
        '"':'||Quotation_Mark||',
        ';':'||Semicolon||',
        '!':'||Exclamation_Mark||',
        '?':'||Question_mark||',
        '(':'||Left_Parentheses||',
        ')':'||Right_Parentheses||',
        '--':'||Dash||',
        '\n':'||Return||',
             }
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests
from collections import Counter

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, shape=[None, None], name="input")
    targets = tf.placeholder(tf.int32, shape=[None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, shape=None, name="learning_rate")
    # TODO: Implement Function
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [4]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = tf.identity(cell.zero_state(batch_size, dtype=tf.float32), name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)))
    embedded_input = tf.nn.embedding_lookup(embedding, input_data)
    return embedded_input


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [6]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(state, name="final_state")
    return output, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    (outputs, final_state) = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(inputs = outputs,
                                               num_outputs = vocab_size, 
                                               activation_fn = None, 
                                               weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer = tf.zeros_initializer())
        
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [25]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    no_batches = int(len(int_text)/ (batch_size * seq_length))
    #print(no_batches)
    drop_xtra_data = np.array(int_text[:no_batches * (batch_size * seq_length)])
    
    in_reshaped = drop_xtra_data.reshape(-1, seq_length)
    
    shifted_in = np.roll(in_reshaped, -1)
    
    target_reshaped = shifted_in.reshape(-1, seq_length)
    
    batches = np.zeros((no_batches, 2, batch_size, seq_length))
    
    for index in range(0, in_reshaped.shape[0]):
        j = index % no_batches
        i = index // no_batches
        batches[j,0,i,:] = in_reshaped[index,:]
        batches[j,1,i,:] = target_reshaped[index,:]
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

7
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [26]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 128
# Sequence Length
seq_length = 4
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

172
Epoch   0 Batch    0/172   train_loss = 8.852
Epoch   0 Batch   10/172   train_loss = 6.357
Epoch   0 Batch   20/172   train_loss = 6.046
Epoch   0 Batch   30/172   train_loss = 5.849
Epoch   0 Batch   40/172   train_loss = 5.792
Epoch   0 Batch   50/172   train_loss = 5.757
Epoch   0 Batch   60/172   train_loss = 5.750
Epoch   0 Batch   70/172   train_loss = 5.261
Epoch   0 Batch   80/172   train_loss = 5.476
Epoch   0 Batch   90/172   train_loss = 5.410
Epoch   0 Batch  100/172   train_loss = 5.234
Epoch   0 Batch  110/172   train_loss = 5.474
Epoch   0 Batch  120/172   train_loss = 5.316
Epoch   0 Batch  130/172   train_loss = 5.432
Epoch   0 Batch  140/172   train_loss = 5.046
Epoch   0 Batch  150/172   train_loss = 4.869
Epoch   0 Batch  160/172   train_loss = 5.212
Epoch   0 Batch  170/172   train_loss = 5.304
Epoch   1 Batch    8/172   train_loss = 4.660
Epoch   1 Batch   18/172   train_loss = 4.776
Epoch   1 Batch   28/172   train_loss = 4.633
Epoch   1 Batch   38/172   tra

Epoch  10 Batch   80/172   train_loss = 3.320
Epoch  10 Batch   90/172   train_loss = 3.367
Epoch  10 Batch  100/172   train_loss = 3.219
Epoch  10 Batch  110/172   train_loss = 3.452
Epoch  10 Batch  120/172   train_loss = 3.382
Epoch  10 Batch  130/172   train_loss = 3.254
Epoch  10 Batch  140/172   train_loss = 3.206
Epoch  10 Batch  150/172   train_loss = 3.284
Epoch  10 Batch  160/172   train_loss = 3.368
Epoch  10 Batch  170/172   train_loss = 3.331
Epoch  11 Batch    8/172   train_loss = 3.159
Epoch  11 Batch   18/172   train_loss = 3.255
Epoch  11 Batch   28/172   train_loss = 3.046
Epoch  11 Batch   38/172   train_loss = 3.176
Epoch  11 Batch   48/172   train_loss = 3.400
Epoch  11 Batch   58/172   train_loss = 3.312
Epoch  11 Batch   68/172   train_loss = 3.225
Epoch  11 Batch   78/172   train_loss = 3.235
Epoch  11 Batch   88/172   train_loss = 3.404
Epoch  11 Batch   98/172   train_loss = 3.446
Epoch  11 Batch  108/172   train_loss = 3.146
Epoch  11 Batch  118/172   train_l

Epoch  20 Batch  160/172   train_loss = 2.492
Epoch  20 Batch  170/172   train_loss = 2.486
Epoch  21 Batch    8/172   train_loss = 2.283
Epoch  21 Batch   18/172   train_loss = 2.446
Epoch  21 Batch   28/172   train_loss = 2.336
Epoch  21 Batch   38/172   train_loss = 2.361
Epoch  21 Batch   48/172   train_loss = 2.387
Epoch  21 Batch   58/172   train_loss = 2.470
Epoch  21 Batch   68/172   train_loss = 2.487
Epoch  21 Batch   78/172   train_loss = 2.558
Epoch  21 Batch   88/172   train_loss = 2.496
Epoch  21 Batch   98/172   train_loss = 2.561
Epoch  21 Batch  108/172   train_loss = 2.369
Epoch  21 Batch  118/172   train_loss = 2.262
Epoch  21 Batch  128/172   train_loss = 2.256
Epoch  21 Batch  138/172   train_loss = 2.310
Epoch  21 Batch  148/172   train_loss = 2.289
Epoch  21 Batch  158/172   train_loss = 2.442
Epoch  21 Batch  168/172   train_loss = 2.332
Epoch  22 Batch    6/172   train_loss = 2.501
Epoch  22 Batch   16/172   train_loss = 2.426
Epoch  22 Batch   26/172   train_l

Epoch  31 Batch   68/172   train_loss = 1.751
Epoch  31 Batch   78/172   train_loss = 1.921
Epoch  31 Batch   88/172   train_loss = 1.890
Epoch  31 Batch   98/172   train_loss = 2.007
Epoch  31 Batch  108/172   train_loss = 1.788
Epoch  31 Batch  118/172   train_loss = 1.719
Epoch  31 Batch  128/172   train_loss = 1.820
Epoch  31 Batch  138/172   train_loss = 1.954
Epoch  31 Batch  148/172   train_loss = 1.811
Epoch  31 Batch  158/172   train_loss = 1.983
Epoch  31 Batch  168/172   train_loss = 1.852
Epoch  32 Batch    6/172   train_loss = 2.010
Epoch  32 Batch   16/172   train_loss = 1.804
Epoch  32 Batch   26/172   train_loss = 1.784
Epoch  32 Batch   36/172   train_loss = 1.855
Epoch  32 Batch   46/172   train_loss = 1.773
Epoch  32 Batch   56/172   train_loss = 1.813
Epoch  32 Batch   66/172   train_loss = 1.847
Epoch  32 Batch   76/172   train_loss = 1.979
Epoch  32 Batch   86/172   train_loss = 2.030
Epoch  32 Batch   96/172   train_loss = 1.809
Epoch  32 Batch  106/172   train_l

Epoch  41 Batch  158/172   train_loss = 1.663
Epoch  41 Batch  168/172   train_loss = 1.608
Epoch  42 Batch    6/172   train_loss = 1.708
Epoch  42 Batch   16/172   train_loss = 1.626
Epoch  42 Batch   26/172   train_loss = 1.566
Epoch  42 Batch   36/172   train_loss = 1.583
Epoch  42 Batch   46/172   train_loss = 1.445
Epoch  42 Batch   56/172   train_loss = 1.573
Epoch  42 Batch   66/172   train_loss = 1.616
Epoch  42 Batch   76/172   train_loss = 1.573
Epoch  42 Batch   86/172   train_loss = 1.568
Epoch  42 Batch   96/172   train_loss = 1.558
Epoch  42 Batch  106/172   train_loss = 1.447
Epoch  42 Batch  116/172   train_loss = 1.656
Epoch  42 Batch  126/172   train_loss = 1.591
Epoch  42 Batch  136/172   train_loss = 1.549
Epoch  42 Batch  146/172   train_loss = 1.716
Epoch  42 Batch  156/172   train_loss = 1.758
Epoch  42 Batch  166/172   train_loss = 1.629
Epoch  43 Batch    4/172   train_loss = 1.527
Epoch  43 Batch   14/172   train_loss = 1.511
Epoch  43 Batch   24/172   train_l

Epoch  52 Batch   66/172   train_loss = 1.455
Epoch  52 Batch   76/172   train_loss = 1.479
Epoch  52 Batch   86/172   train_loss = 1.441
Epoch  52 Batch   96/172   train_loss = 1.479
Epoch  52 Batch  106/172   train_loss = 1.371
Epoch  52 Batch  116/172   train_loss = 1.525
Epoch  52 Batch  126/172   train_loss = 1.431
Epoch  52 Batch  136/172   train_loss = 1.344
Epoch  52 Batch  146/172   train_loss = 1.520
Epoch  52 Batch  156/172   train_loss = 1.556
Epoch  52 Batch  166/172   train_loss = 1.508
Epoch  53 Batch    4/172   train_loss = 1.357
Epoch  53 Batch   14/172   train_loss = 1.405
Epoch  53 Batch   24/172   train_loss = 1.480
Epoch  53 Batch   34/172   train_loss = 1.327
Epoch  53 Batch   44/172   train_loss = 1.589
Epoch  53 Batch   54/172   train_loss = 1.379
Epoch  53 Batch   64/172   train_loss = 1.400
Epoch  53 Batch   74/172   train_loss = 1.367
Epoch  53 Batch   84/172   train_loss = 1.364
Epoch  53 Batch   94/172   train_loss = 1.425
Epoch  53 Batch  104/172   train_l

Epoch  62 Batch  146/172   train_loss = 1.467
Epoch  62 Batch  156/172   train_loss = 1.408
Epoch  62 Batch  166/172   train_loss = 1.367
Epoch  63 Batch    4/172   train_loss = 1.302
Epoch  63 Batch   14/172   train_loss = 1.325
Epoch  63 Batch   24/172   train_loss = 1.441
Epoch  63 Batch   34/172   train_loss = 1.263
Epoch  63 Batch   44/172   train_loss = 1.471
Epoch  63 Batch   54/172   train_loss = 1.308
Epoch  63 Batch   64/172   train_loss = 1.399
Epoch  63 Batch   74/172   train_loss = 1.359
Epoch  63 Batch   84/172   train_loss = 1.365
Epoch  63 Batch   94/172   train_loss = 1.441
Epoch  63 Batch  104/172   train_loss = 1.347
Epoch  63 Batch  114/172   train_loss = 1.346
Epoch  63 Batch  124/172   train_loss = 1.332
Epoch  63 Batch  134/172   train_loss = 1.236
Epoch  63 Batch  144/172   train_loss = 1.350
Epoch  63 Batch  154/172   train_loss = 1.446
Epoch  63 Batch  164/172   train_loss = 1.386
Epoch  64 Batch    2/172   train_loss = 1.402
Epoch  64 Batch   12/172   train_l

Epoch  73 Batch   54/172   train_loss = 1.257
Epoch  73 Batch   64/172   train_loss = 1.296
Epoch  73 Batch   74/172   train_loss = 1.321
Epoch  73 Batch   84/172   train_loss = 1.302
Epoch  73 Batch   94/172   train_loss = 1.442
Epoch  73 Batch  104/172   train_loss = 1.325
Epoch  73 Batch  114/172   train_loss = 1.318
Epoch  73 Batch  124/172   train_loss = 1.352
Epoch  73 Batch  134/172   train_loss = 1.288
Epoch  73 Batch  144/172   train_loss = 1.281
Epoch  73 Batch  154/172   train_loss = 1.416
Epoch  73 Batch  164/172   train_loss = 1.337
Epoch  74 Batch    2/172   train_loss = 1.339
Epoch  74 Batch   12/172   train_loss = 1.174
Epoch  74 Batch   22/172   train_loss = 1.269
Epoch  74 Batch   32/172   train_loss = 1.447
Epoch  74 Batch   42/172   train_loss = 1.407
Epoch  74 Batch   52/172   train_loss = 1.286
Epoch  74 Batch   62/172   train_loss = 1.401
Epoch  74 Batch   72/172   train_loss = 1.342
Epoch  74 Batch   82/172   train_loss = 1.313
Epoch  74 Batch   92/172   train_l

Epoch  83 Batch  134/172   train_loss = 1.237
Epoch  83 Batch  144/172   train_loss = 1.327
Epoch  83 Batch  154/172   train_loss = 1.436
Epoch  83 Batch  164/172   train_loss = 1.371
Epoch  84 Batch    2/172   train_loss = 1.349
Epoch  84 Batch   12/172   train_loss = 1.188
Epoch  84 Batch   22/172   train_loss = 1.218
Epoch  84 Batch   32/172   train_loss = 1.350
Epoch  84 Batch   42/172   train_loss = 1.350
Epoch  84 Batch   52/172   train_loss = 1.237
Epoch  84 Batch   62/172   train_loss = 1.377
Epoch  84 Batch   72/172   train_loss = 1.322
Epoch  84 Batch   82/172   train_loss = 1.260
Epoch  84 Batch   92/172   train_loss = 1.215
Epoch  84 Batch  102/172   train_loss = 1.310
Epoch  84 Batch  112/172   train_loss = 1.207
Epoch  84 Batch  122/172   train_loss = 1.409
Epoch  84 Batch  132/172   train_loss = 1.207
Epoch  84 Batch  142/172   train_loss = 1.437
Epoch  84 Batch  152/172   train_loss = 1.263
Epoch  84 Batch  162/172   train_loss = 1.312
Epoch  85 Batch    0/172   train_l

Epoch  94 Batch   42/172   train_loss = 1.378
Epoch  94 Batch   52/172   train_loss = 1.264
Epoch  94 Batch   62/172   train_loss = 1.382
Epoch  94 Batch   72/172   train_loss = 1.296
Epoch  94 Batch   82/172   train_loss = 1.276
Epoch  94 Batch   92/172   train_loss = 1.264
Epoch  94 Batch  102/172   train_loss = 1.280
Epoch  94 Batch  112/172   train_loss = 1.266
Epoch  94 Batch  122/172   train_loss = 1.441
Epoch  94 Batch  132/172   train_loss = 1.260
Epoch  94 Batch  142/172   train_loss = 1.471
Epoch  94 Batch  152/172   train_loss = 1.234
Epoch  94 Batch  162/172   train_loss = 1.337
Epoch  95 Batch    0/172   train_loss = 1.353
Epoch  95 Batch   10/172   train_loss = 1.358
Epoch  95 Batch   20/172   train_loss = 1.271
Epoch  95 Batch   30/172   train_loss = 1.442
Epoch  95 Batch   40/172   train_loss = 1.355
Epoch  95 Batch   50/172   train_loss = 1.330
Epoch  95 Batch   60/172   train_loss = 1.299
Epoch  95 Batch   70/172   train_loss = 1.318
Epoch  95 Batch   80/172   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [34]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.